# Taller 3

**Autor:** Juan Pablo Gaviria

## Deteccion de Topicos con genSim

Se realizará la detección de Topicos con genSim y LDA

In [ ]:
import pandas as pd
import numpy as np
from gensim.corpora import Dictionary
import snowballstemmer
from nltk.corpus import stopwords
from string import ascii_lowercase
import itertools
from collections import Counter
import re
import multiprocessing as mp
import time
from gensim.models.ldamulticore import LdaMulticore
import pyLDAvis
import gensim
import nltk
from pyLDAvis import gensim

### Lectura de la informacion

In [ ]:
originalData = pd.read_csv('/home/jgaviria/Workspace/DataScience/datasets/metadata.csv')
originalData

### Preparacion de la informacion

In [ ]:
data = originalData.drop(['sha','doi','pmcid','pubmed_id','license','mag_id','who_covidence_id','arxiv_id','pdf_json_files','pmc_json_files','url','s2_id'], axis=1)
data

In [ ]:
# Se crea nuevo dataframe unicamente con el id y el texto a evaluar
texto = pd.DataFrame({'id':data.cord_uid, 'texto':data.title+" "+data.abstract})
texto

In [ ]:
# Se eliminan filas que no tengan ni titulo ni abstract
texto = texto.dropna()
texto

### Preparacion del texto

In [ ]:
# Preparacion stopwords
# Se crea un listado de stopwords con nltk y se expande a su raiz con stem
stemmer = snowballstemmer.EnglishStemmer()
stop = stopwords.words('english')
stop.extend(['may','also','zero','one','two','three','four','five','six','seven','eight','nine','ten','across','among','beside','however','yet','within']+list(ascii_lowercase))
stopList = stemmer.stemWords(stop)
stopList = set(stopList)
stop = set(sorted(stop + list(stopList)))

In [ ]:
# Limpieza de caracteres especiales
texto['texto'].replace('[!"#%\'()*+,-./:;<=>?@\[\]^_`{|}~1234567890’”“′‘\\\]',' ',inplace=True,regex=True)

In [ ]:
# Tokenizaicion, Stemming y remocion de stopwords
wordlist = filter(None, " ".join(list(set(list(itertools.chain(*texto['texto'].str.split(' ')))))).split(" "))
texto['stemmed_text'] = [' '.join(filter(None,filter(lambda word: word not in stop, line))) for line in texto['texto'].str.lower().str.split(' ')]
texto



In [ ]:
# Se eliminan las palabras que no aparezcan por lo menos 5 veces en todos los documentos
cuenta_minima = 5
str_frequencies = pd.DataFrame(list(Counter(filter(None,list(itertools.chain(*texto['stemmed_text'].str.split(' '))))).items()),columns=['word','count'])
low_frequency_words = set(str_frequencies[str_frequencies['count'] < cuenta_minima]['word'])
texto['stemmed_text'] = [' '.join(filter(None,filter(lambda word: word not in low_frequency_words, line))) for line in texto['stemmed_text'].str.split(' ')]
texto

In [ ]:
# Funcion para Stemming paralelo
def parStemming(texto):
    return " ".join(stemmer.stemWords(re.sub('[!"#%\'()*+,-./:;<=>?@\[\]^_`{|}~1234567890’”“′‘\\\]',' ', texto).split(' ')))

#texto['stemmed_text'] = [" ".join(stemmer.stemWords(re.sub('[!"#%\'()*+,-./:;<=>?@\[\]^_`{|}~1234567890’”“′‘\\\]',' ', next_text).split(' '))) for next_text in texto['stemmed_text']]

In [ ]:
# Se realiza Stemming
t0 = time.time()
pool = mp.Pool()
texto['stemmed_text'] = pool.map(parStemming,texto['stemmed_text'])
pool.close()
print('Tiempo tomado: ' + str(time.time()-t0))

### Representacion de los documentos

In [ ]:
# Funcion para tokenizacion paralela
def parToken(texto):
    return nltk.word_tokenize(texto)

In [ ]:
# Tokenizacion
t0 = time.time()
pool = mp.Pool()
#texto['tokens'] = texto.apply(lambda row: nltk.word_tokenize(row['stemmed_text']), axis=1)
texto['tokens'] = pool.map(parToken, texto['stemmed_text'])
pool.close()
print('Tiempo tomado: ' + str(time.time()-t0))
texto

In [ ]:
# BoW
diccionario = Dictionary(texto.tokens)
print('Longitud del BoW: ' + str(len(diccionario)))

In [ ]:
# Matriz de documentos vs terminos
t0 = time.time()
pool = mp.Pool(mp.cpu_count()-1)
doc_term_matrix = pool.map(diccionario.doc2bow, [sentence for sentence in texto.tokens])
pool.close()
print('Tiempo tomado: '+str(time.time()-t0))

### Aplicacion del modelo (LDA)

In [ ]:
t0 = time.time()
lda_model = LdaMulticore(doc_term_matrix, num_topics=5, id2word=diccionario, workers=mp.cpu_count())
print('Tiempo tomado: '+str(time.time()-t0))

### Visualizacion del modelo

In [ ]:
t0 = time.time()
pyLDAvis.enable_notebook()
vis = gensim.prepare(lda_model, doc_term_matrix, diccionario, sort_topics=False)
print('Tiempo tomado: '+str(time.time()-t0))
vis

In [ ]:
# Guardar el modelo
pyLDAvis.save_html(vis, 'gensimLDAOpt.html')

### Aplicación del modelo

Se aplica el modelo para clasificar todos los documentos. Esta clasificacion se utiliza para el clasificador supervisado

In [ ]:
# Funcion para clasificacion paralela
def clasificarDoc(documento):
    textBow = diccionario.doc2bow(documento)
    distribucion = lda_model.get_document_topics(textBow)
    topico = max(distribucion, key=lambda x:x[1])[0]
    return topico


In [ ]:
# Clasificar todos los documentos
t0 = time.time()
pool = mp.Pool()
texto['Topico'] = pool.map(clasificarDoc, texto.tokens)
pool.close()
print('Tiempo tomado: '+str(time.time()-t0))
texto

In [ ]:
# Se crea Data Frame de salida con el id del documento y el topico
outClasificacion = pd.DataFrame({'docId':texto.id,'topico':texto.Topico})
outClasificacion.to_pickle('docTopicDetection.pkl')